In [1]:
import pandas as pd
import numpy as np
import itertools
import random
import ast
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import spearmanr, wilcoxon, friedmanchisquare, kendalltau, pearsonr
from matplotlib import animation
from mpl_toolkits import mplot3d
from scipy.spatial import distance
from pyxdameraulevenshtein import damerau_levenshtein_distance

pd.set_option('display.max_rows', 500)

### Prepare Data

In [2]:
# read data

df = pd.read_csv('automatica_task_environments_clustered_drawers.csv', header=0,
                 skiprows=[29,49,57,66], skip_blank_lines=True,
                usecols=['ID', 'sequence_clustered_silverware', 'coordinates_clustered_silverware',
                        'strong_k', 'mid_k', 'containment', 'error_clustered_silverware',
                        'start_coordinates'],
                nrows=79)

### Define functions for prediction

In [3]:
def predict_sequence(objects, coordinates, start_coordinates, c, k, dimension=[3,]):
    ''' Predicts sequence based on required objects, object coordinates, start coordinates of subject,
        parameters (c+k) and dimensionality.
        Input: Objects, object coordinates, start coordinates, c, k, dimension
        Output: Sequence of objects as str
    '''
    prediction = []
    possible_items = dict.fromkeys(objects, 0) # generate dict from object list
    coord_index = 0
    start_coords = start_coordinates
    coords = coordinates
    new_coords = {}
    new_start_coords = []
    
    if dimension[0] == 3: # no changes if 3D
        new_coords = coords
        new_start_coords = start_coords
        
    elif dimension[0] == 2: # 2D: remove obsolete coordinate
        if dimension[1] == 'xy':
            new_coords = {key: value[:-1] for key, value in coords.items()}
            new_start_coords = [x[:-1] for x in start_coords]
                
        elif dimension[1] == 'xz':
            new_start_coords = [[x[0], x[-1]] for x in start_coords]
            
            for key, value in coords.items():
                new_value = (value[0], value[-1])
                new_coords[key] = new_value
        
        elif dimension[1] == 'yz':
            new_coords = {key: value[1:] for key, value in coords.items()}
            new_start_coords = [x[1:] for x in start_coords]
                
    elif dimension[0] == 1: # 1D: choose appropriate coordinate
        if dimension[1] == 'x':
            new_coords = {key: value[0] for key, value in coords.items()}
            new_start_coords = [x[0] for x in start_coords]
        
        elif dimension[1] == 'y':
            new_coords = {key: value[1] for key, value in coords.items()}
            new_start_coords = [x[1] for x in start_coords]
        
        elif dimension[1] == 'z':
            new_coords = {key: value[2] for key, value in coords.items()}
            new_start_coords = [x[2] for x in start_coords]
    
    while bool(possible_items) == True: # while dict not empty
        for obj in possible_items.keys():
            possible_items[obj] = ((distance.euclidean(
                                new_start_coords[coord_index], 
                                new_coords[obj])
                                ) ** k[obj]) * c[obj]
        #print(possible_items)                      
        minval = min(possible_items.values())
        minval = [k for k, v in possible_items.items() if v == minval]
        minval = random.choice(minval) # choose prediction randomly if multiple items have same cost
        prediction.append(minval)
        del possible_items[minval]
        coord_index += 1
        
    return prediction

In [4]:
def get_median(objects, coordinates, start_coordinates, c, k, dimension, sequence,n=10):
    ''' Returns average edit distance (Damerau-Levenshtein) for 100 trials of sequence prediction.
    '''
    edit_list = []

    for x in range(0,n):
        #print(objects)
        result = ''.join(predict_sequence(objects, coordinates, start_coordinates, c, k, dimension))
        dl = damerau_levenshtein_distance(sequence,result)
        edit_list.append(dl)
    
    #avg = np.mean(edit_list)
    median = np.median(edit_list)
    return median

In [5]:
def get_avg_editdist(data, dimensions=[[1,'x'],[1,'y'],[1,'z'],[2,'xy'],[2,'xz'],[2,'yz'],[3,'xyz']],n=10):
    ''' Calculates average edit distance for all combinations of parameters (c, k, dimension).
        Input: Dataframe with objects, coordinates, start coordinates, object categories
        Output: Dataframe with edit distance results (col name: parameters used)
    '''
    results = pd.DataFrame()
        
    for row in range(0,len(data)):
        objects = list(data.at[row,'sequence_clustered_silverware'])
        strong_k = list(data.at[row,'strong_k'].split(','))
        mid_k = list(data.at[row,'mid_k'].split(','))
        #food_k = list(data.at[row,'food_k'].split(','))
        food_k = []
        coordinates = {key: ast.literal_eval(value) for key, value in (elem.split(': ') for elem in data.at[row,'coordinates_clustered_silverware'].split(';'))}
        #print(coordinates)
        start_coordinates = list(ast.literal_eval(data.at[row, 'start_coordinates']))
        sequence = str(data.at[row,'sequence_clustered_silverware'])
        c1 = {obj: 1.2 for obj in objects}
        k1 = {obj: 1.0 for obj in objects}
        
        #for k2 in np.arange(1.1,2.0,0.1):
            #k_food = round(k2,2)
            #k1 = {obj: k_food if obj in food_k else 1.0 for obj in objects}
        
        for k in np.arange(0,0.9,0.1):
            k_strong = round(k,2)
            k_mid = round(k + 0.1,2)
            k1 = {obj: k_strong if obj in strong_k else k_mid if obj in mid_k else round(k1[obj],2) for obj in objects}
            
                #for c in np.arange(1.0,2.0,0.1):
                #    c = round(c, 1)
                #    c1 = {obj: 1.2 for obj in objects}
                
            for dim in dimensions:                
                # get average edit distance
                median = get_median(objects, coordinates, start_coordinates, c1, k1, dim, sequence, n)
                median = median / len(sequence)
                #params = 'c: ' + str(c) + '; k: ' + str(k_strong) + ',' + str(k_mid) + ',' + str(k_food) + '; ' + str(dim[1])
                params = 'c: 1.2' + '; k: ' + str(k_strong) + ',' + str(k_mid) + '; ' + str(dim[1])
                results.at[row,params] = median
                        
        results.at[row,'sequence'] = sequence
        results.at[row,'error'] = data.at[row,'error_clustered_silverware']
        results.at[row,'ID'] = data.at[row,'ID']
                    
    return results

### Calculate edit distances, create df

In [6]:
# ~3 min for n=100
%time results_new = get_avg_editdist(df,n=100)

CPU times: user 2min 30s, sys: 0 ns, total: 2min 30s
Wall time: 2min 30s


In [7]:
#results_new

### Get lowest error, compare edit distances

In [29]:
def get_lowest_error(results):
    ''' Returns lowest error in dataframe and index of lowest error.
    '''
    for col in list(results):
        if col != 'sequence' and col != 'ID':
            results.loc['mean',col] = results[col].mean()
            #results.loc['median',col] = results[col].median()
    lowest = min(results.loc['mean'])
    mean = list(results.loc['mean'])
    
    return lowest, results.columns[(results.loc['mean'] == lowest)], mean, results

In [30]:
lowest, lowest_idx, list_mean, results_mean = get_lowest_error(results_new)
lowest, lowest_idx

results_mean

,"c: 1.2; k: 0.0,0.1; x","c: 1.2; k: 0.0,0.1; y","c: 1.2; k: 0.0,0.1; z","c: 1.2; k: 0.0,0.1; xy","c: 1.2; k: 0.0,0.1; xz","c: 1.2; k: 0.0,0.1; yz","c: 1.2; k: 0.0,0.1; xyz","c: 1.2; k: 0.1,0.2; x","c: 1.2; k: 0.1,0.2; y","c: 1.2; k: 0.1,0.2; z",...,"c: 1.2; k: 0.8,0.9; x","c: 1.2; k: 0.8,0.9; y","c: 1.2; k: 0.8,0.9; z","c: 1.2; k: 0.8,0.9; xy","c: 1.2; k: 0.8,0.9; xz","c: 1.2; k: 0.8,0.9; yz","c: 1.2; k: 0.8,0.9; xyz",sequence,error,ID
0,0.800000,0.600000,1.000000,0.600000,0.800000,0.600000,0.600000,0.800000,0.600000,1.000000,...,0.600000,0.600000,0.800000,0.400000,0.600000,0.600000,0.400000,pocgz,0.666000,a1
1,0.666667,1.000000,0.666667,1.000000,0.666667,1.000000,1.000000,0.666667,1.000000,0.833333,...,0.666667,1.000000,1.000000,1.000000,0.666667,1.000000,1.000000,cgwpdz,0.723000,a3
2,0.250000,0.500000,1.000000,0.250000,1.000000,0.500000,0.250000,0.250000,0.500000,1.000000,...,0.000000,0.250000,0.500000,0.000000,0.500000,0.250000,0.000000,zpwg,0.605000,a5
3,0.777778,0.777778,0.888889,0.666667,0.777778,0.777778,0.666667,0.777778,0.777778,0.833333,...,0.777778,0.666667,0.777778,0.555556,0.777778,0.666667,0.555556,pzwzocndg,0.806000,a11
4,0.857143,1.000000,0.857143,0.857143,0.857143,1.000000,0.857143,0.857143,0.857143,0.857143,...,0.714286,0.857143,0.571429,0.857143,0.714286,0.857143,0.857143,wptgzoc,0.756000,a13
5,0.500000,0.500000,0.500000,0.250000,0.500000,0.500000,0.250000,0.500000,0.500000,0.500000,...,0.500000,1.000000,0.250000,0.500000,0.500000,1.000000,0.500000,pwzg,0.605000,a16
6,0.800000,0.600000,0.800000,0.600000,1.000000,0.600000,0.600000,0.800000,0.600000,0.800000,...,0.600000,0.600000,0.800000,0.600000,0.600000,0.600000,0.600000,powcz,0.666000,a19
7,0.750000,0.500000,0.500000,0.500000,0.750000,0.500000,0.500000,0.750000,0.500000,0.500000,...,0.750000,0.500000,0.500000,0.750000,0.750000,0.500000,0.750000,copz,0.605000,a22
8,1.000000,0.600000,0.600000,0.600000,1.000000,0.600000,0.600000,1.000000,0.600000,0.600000,...,0.800000,0.400000,0.600000,0.600000,0.800000,0.400000,0.600000,wopnz,0.666000,a26
9,0.888889,0.777778,0.777778,0.777778,0.888889,0.666667,0.777778,0.888889,0.777778,0.888889,...,0.888889,0.777778,0.777778,0.777778,0.888889,0.666667,0.777778,opwqxgcnd,0.806000,a29


In [23]:
# stdev for results better than baseline
np.std(results_new['c: 1.2; k: 0.8,0.9; xy'].loc[results_new['c: 1.2; k: 0.8,0.9; xy'] <= results_new['error']])

0.16786928651063757

In [39]:
# avg difference between best model and error
np.mean(results_new['error'][:-1] - results_new['c: 1.2; k: 0.8,0.9; xy'][:-1])

0.06903964390040336

In [36]:
results_new.loc['mean','error'], results_new.loc['mean','c: 1.2; k: 0.8,0.9; xy']

(0.6931772151898734, 0.62413757128947)

In [46]:
np.nanmean(list_mean), np.nanmedian(list_mean)

(0.6802036283877755, 0.6639004806726325)

In [47]:
# Compare avg edit distance for xy and xyz
list_xy = []
list_xyz = []

for col in results_mean:
    if col != 'sequence' and col != 'error' and col != 'ID' and col.split(';')[2].strip() == 'xy':
        list_xy.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'error' and col != 'ID' and col.split(';')[2].strip() == 'xyz':
        list_xyz.append(results_mean.at['mean',col])
        
avg_xy = np.mean(list_xy)
med_xy = np.median(list_xy)
std_xy = np.std(list_xy)
print('Average xy: ' + str(avg_xy)  + ', stdev: ' + str(std_xy) + ', median: ' + str(med_xy))
        
avg_xyz = np.mean(list_xyz)
med_xyz = np.median(list_xyz)
std_xyz = np.std(list_xyz)
print('Average xyz: ' + str(avg_xyz) + ', stdev: ' + str(std_xyz) + ', median: ' + str(med_xyz))

Average xy: 0.6385630739006266, stdev: 0.01095465579576928, median: 0.6364190661659016
Average xyz: 0.6410043035570462, stdev: 0.012996713789192522, median: 0.6391616821996569


In [48]:
# Compare avg edit distances for x, y, z
list_x = []
list_y = []
list_z = []
list_xz = []
list_yz = []

for col in results_mean:
    if col != 'sequence' and col != 'error' and col != 'ID' and col.split(';')[2].strip() == 'x':
        list_x.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'error' and col != 'ID' and col.split(';')[2].strip() == 'y':
        list_y.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'error' and col != 'ID' and col.split(';')[2].strip() == 'z':
        list_z.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'error' and col != 'ID' and col.split(';')[2].strip() == 'xz':
        list_xz.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'error' and col != 'ID' and col.split(';')[2].strip() == 'yz':
        list_yz.append(results_mean.at['mean',col])

avg_xz = np.mean(list_xz)
avg_yz = np.mean(list_yz)
        
avg_x = np.mean(list_x)
lowest_x = min(list_x)
print('Average x: ' + str(avg_x) + ', min: ' + str(lowest_x)) 
        
avg_y = np.mean(list_y)
lowest_y = min(list_y)
print('Average y: ' + str(avg_y) + ', min: ' + str(lowest_y)) 
        
avg_z = np.mean(list_z)
lowest_z = min(list_z)
print('Average z: ' + str(avg_z) + ', min: ' + str(lowest_z)) 

print('Average xz: ' + str(avg_xz)) 
print('Average yz: ' + str(avg_yz)) 

Average x: 0.7086022197836542, min: 0.6532615415526808
Average y: 0.667868326781829, min: 0.6407130492573531
Average z: 0.7484336924210342, min: 0.6884626976399129
Average xz: 0.6948439316582777
Average yz: 0.6621098506119604


### Statistical analysis

In [49]:
# Compare all error predictions for dimensions
stat, p = friedmanchisquare(list_x,list_y,list_z,list_xy,list_xz,list_yz,list_xyz)
print('Friedman: stat = %.3f, p = % 10.3E' % (stat, p))

Friedman: stat = 38.619, p =  8.500E-07


In [50]:
# 2D vs 3D median
stat, p = wilcoxon(list_xz, list_xy, zero_method='wilcox')
print('Wilcoxon: W = %.3f, p = %.5f' % (stat, p))

Wilcoxon: W = 4.000, p = 0.02734


### Plots for dataframe

In [51]:
# Define input for plots

c = [float(x[3:6]) for x in results_new.columns.tolist() if x != 'sequence' and x != 'error' and x != 'ID']
#k = [float(x[11:14]) for x in results_new.columns.tolist() if x != 'sequence']
k_strong = [float(x[11:14]) for x in results_new.columns.tolist() if x != 'sequence' and x != 'error' and x != 'ID']
k_mid = [float(x[15:18]) for x in results_new.columns.tolist() if x != 'sequence' and x != 'error' and x != 'ID']
dim = [x.strip() for x in results_new.columns[:-3].str.split(';').str[2]]
median = [x for x in results_mean.loc['mean'][:-1].tolist() if pd.notnull(x) and x != 'error' and x != 'ID']

In [52]:
%matplotlib qt

cm = ['red','blue','green','magenta','cyan','orange','grey']
dim_num = [0 if x=='x' else 1 if x=='y' else 2 if x=='z' else 3 if x=='xy' else 4 if x=='xz' else 5 if x=='yz' else 6 for x in dim]

cmap = matplotlib.colors.ListedColormap(cm)

ticks = ['x', 'y', 'z', 'xy', 'xz', 'yz', 'xyz']
norm = matplotlib.colors.BoundaryNorm(ticks, cmap.N)

# create figure, 3d grid, set background to white
fig2 = plt.figure(figsize=(12,8))
ax2 = fig2.add_subplot(111, projection='3d')
ax2.w_xaxis.set_pane_color((1.0,1.0,1.0,1.0))
ax2.w_yaxis.set_pane_color((1.0,1.0,1.0,1.0))
ax2.w_zaxis.set_pane_color((1.0,1.0,1.0,1.0))

# create plot w/ median edit distance
#img = ax2.scatter(c, k, median, alpha=0.5, s=38, c=dim_num, cmap=cmap)
img = ax2.scatter(k_strong, k_mid, median, alpha=0.5, s=38, c=dim_num, cmap=cmap)

# plot horizontal plane for baseline
#xx, yy = np.meshgrid(np.linspace(1,1.9), np.linspace(0,0.9))
xx, yy = np.meshgrid(np.linspace(0,0.9), np.linspace(0,0.9))
#yy = np.meshgrid(range(2), range(2))
zz = xx * 0 + 0.706
ax2.plot_surface(xx, yy, zz, alpha=0.5)

# set labels
ax2.set_ylabel('strong k', fontsize=20, labelpad=10)
ax2.set_xlabel('mid k', fontsize=20, labelpad=10)
ax2.set_zlabel('normalized edit distance', fontsize=20, labelpad=7)
#plt.title('Average edit distance (1 step)', fontsize=24)

# create colorbar
cb = plt.colorbar(img, cax = fig2.add_axes([0.9,0.3,0.03,0.4]))
cb.ax.set_yticklabels(ticks, fontsize=16)
plt.show()

/home/zauri/files/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: UserWarning: FixedFormatter should only be used together with FixedLocator


### Check for sequences worse than baseline

In [62]:
# get sequences from best model where performance worse than baseline
higher_error = results_new['c: 1.2; k: 0.8,0.9; xy'].loc[results_new['c: 1.2; k: 0.8,0.9; xy'] >= results_new['error']]

print('Higher than avg. error: ' +
    str(len(higher_error))
    + ' of ' + \
    str(len(results_new['c: 1.2; k: 0.8,0.9; xy'])) + ' sequences')

Higher than avg. error: 24 of 80 sequences


In [61]:
worse_than_baseline = results_new['ID'].loc[results_new['c: 1.2; k: 0.8,0.9; xy'] >= results_new['error']]

worse_than_baseline

1       a3
4      a13
7      a22
11     a35
14     a40
20     a56
21     a59
24     a69
25     a73
29     a85
36    a106
40    a116
41    a118
43    a128
44    a131
45    a137
55    a174
57    a178
59    a185
61    a191
67    a215
70    a222
72    a226
73    a229
Name: ID, dtype: object

In [57]:
#%matplotlib inline

error_individual = df['error_clustered_silverware']
seqs = results_mean['sequence'][:-1].values
res = results_new['c: 1.2; k: 0.7,0.8; xy'][:-1].values
x = [x for x in range (0,len(seqs))]

plt.figure(figsize=(18,6))
plt.scatter(x, res, marker='o', label='model-generated sequences')
plt.scatter(x, error_individual, c='red', marker='o', linewidth=2, label='baseline for sequence length')
plt.plot(x, error_individual, c='red', alpha=0.5)
plt.plot((x,x),(error_individual,res), '--', c='black', alpha=0.6)


plt.title('silverware clustered for silverware, 0.624, c: 1.2; k: 0.8,0.9; xy', fontsize=16)
plt.xticks(x, seqs, rotation=90)
plt.ylabel('median edit distance', fontsize=14)
plt.xlabel('sequence', fontsize=14)
plt.legend()

#plt.savefig('plot_median_editdist_individualerrors_diff.png', bbox_inches='tight')
plt.show()